In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
from memory_profiler import memory_usage

import rpy2.rinterface
import dask.dataframe as dd
## install the packages https://arrow.apache.org/docs/python/install.html
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
## How to install put instructions https://anaconda.org/conda-forge/rpy2
import rpy2.rinterface
# install this https://pypi.org/project/rpy2-arrow/#description  pip install rpy2-arrow
# have to install this as well conda install -c conda-forge r-arrow 
import rpy2_arrow.pyarrow_rarrow as pyra
### instruction
import pyarrow.feather as feather

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## 1. Download the data

In [3]:
#global variables

# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"
# Link up with figshare
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
# Download desired files
files_to_dl = ["README.md", "data.zip"]       # list of files to download
os.makedirs(output_directory, exist_ok=True)  # output directory to save in
for file in files:
    if file["name"] in files_to_dl:
        urlretrieve(file["download_url"], output_directory + file["name"])

In [4]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 19.4 s, sys: 2.95 s, total: 22.4 s
Wall time: 23.9 s


# 2. Combinning data CSVs

In [5]:
df = pd.read_csv("figshare/INM-CM4-8_daily_rainfall_NSW.csv", index_col=0, parse_dates=True)
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
time,,,,,
1889-01-01 12:00:00,-36.0,-34.5,141.0,143.0,3.176243e-08
1889-01-02 12:00:00,-36.0,-34.5,141.0,143.0,3.678816e-09
1889-01-03 12:00:00,-36.0,-34.5,141.0,143.0,2.017436e-07
1889-01-04 12:00:00,-36.0,-34.5,141.0,143.0,3.976414e+00
1889-01-05 12:00:00,-36.0,-34.5,141.0,143.0,2.978595e+00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1609650 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   lat_min        1609650 non-null  float64
 1   lat_max        1609650 non-null  float64
 2   lon_min        1609650 non-null  float64
 3   lon_max        1609650 non-null  float64
 4   rain (mm/day)  1609650 non-null  float64
dtypes: float64(5)
memory usage: 73.7 MB


In [7]:
files = glob.glob('figshare/*_daily_rainfall_NSW.csv')

In [8]:
for f in files:
    print(f)

figshare/MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
figshare/AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
figshare/NorESM2-LM_daily_rainfall_NSW.csv
figshare/ACCESS-CM2_daily_rainfall_NSW.csv
figshare/FGOALS-f3-L_daily_rainfall_NSW.csv
figshare/CMCC-CM2-HR4_daily_rainfall_NSW.csv
figshare/MRI-ESM2-0_daily_rainfall_NSW.csv
figshare/GFDL-CM4_daily_rainfall_NSW.csv
figshare/BCC-CSM2-MR_daily_rainfall_NSW.csv
figshare/EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
figshare/CMCC-ESM2_daily_rainfall_NSW.csv
figshare/NESM3_daily_rainfall_NSW.csv
figshare/MPI-ESM1-2-LR_daily_rainfall_NSW.csv
figshare/ACCESS-ESM1-5_daily_rainfall_NSW.csv
figshare/FGOALS-g3_daily_rainfall_NSW.csv
figshare/INM-CM4-8_daily_rainfall_NSW.csv
figshare/MPI-ESM1-2-HR_daily_rainfall_NSW.csv
figshare/TaiESM1_daily_rainfall_NSW.csv
figshare/NorESM2-MM_daily_rainfall_NSW.csv
figshare/CMCC-CM2-SR5_daily_rainfall_NSW.csv
figshare/KIOST-ESM_daily_rainfall_NSW.csv
figshare/INM-CM5-0_daily_rainfall_NSW.csv
figshare/MIROC6_daily_rainfall_NSW.csv


In [9]:
%%time
%memit

df = pd.concat((pd.read_csv(file, index_col=0, parse_dates=True).assign(model = re.findall(r'(?<=figshare/).+(?=_daily_rainfall_NSW)', file)[0]) for file in files))

peak memory: 320.26 MiB, increment: 0.09 MiB
CPU times: user 1min 16s, sys: 13.4 s, total: 1min 29s
Wall time: 1min 37s


In [10]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB


In [12]:
df.to_csv("figshare/combined_data.csv")

In [13]:
%%sh
du -sh figshare/combined_data.csv

4.4G	figshare/combined_data.csv


In [14]:
%%time
%%memit
# shows time that dask take to merge

ddf = dd.read_csv("figshare/*.csv",assume_missing=True)

peak memory: 1502.21 MiB, increment: 9.68 MiB
CPU times: user 181 ms, sys: 1.18 s, total: 1.36 s
Wall time: 3.25 s


In [15]:
ddf.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02


# 3. Load the combined CSV to memory and find the maximum rain drop(Python)

In [5]:
%%time
%%memit

max_rain_slow = pd.read_csv("figshare/combined_data.csv")['rain (mm/day)'].max()
print(f'the max rain drop is: {max_rain_slow} mm/day')

the max rain drop is: 432.9395145177841 mm/day
peak memory: 3517.64 MiB, increment: 3185.21 MiB
CPU times: user 54.8 s, sys: 15.6 s, total: 1min 10s
Wall time: 1min 15s


In [19]:
%%time
%%memit
max_rain = np.finfo('float64').min

for chunk in pd.read_csv('figshare/combined_data.csv', chunksize=10_000_000):
    m = chunk['rain (mm/day)'].max()
    if m > max_rain:
        max_rain = m

print(f'the max rain drop is: {max_rain} mm/day')

the max rain drop is: 432.9395145177841
peak memory: 2224.97 MiB, increment: 1549.34 MiB
CPU times: user 54.2 s, sys: 6.9 s, total: 1min 1s
Wall time: 1min 4s


In [3]:
%%time
%%memit

ddf = dd.read_csv('figshare/combined_data.csv')
max_rain_dd = ddf['rain (mm/day)'].max().compute()


peak memory: 1019.96 MiB, increment: 866.99 MiB
CPU times: user 1min 14s, sys: 13.9 s, total: 1min 28s
Wall time: 44.9 s


In [4]:
print(f'the max rain drop is: {max_rain_dd} mm/day')

the max rain drop is: 432.9395145177841 mm/day


In [3]:
%%time
%%memit

part_df = pd.read_csv('figshare/combined_data.csv', usecols=['rain (mm/day)'])
max_rain_part = part_df['rain (mm/day)'].max()
print(f'the max rain drop is: {max_rain_part} mm/day')

the max rain drop is: 432.9395145177841 mm/day
peak memory: 688.32 MiB, increment: 452.52 MiB
CPU times: user 23.6 s, sys: 2.6 s, total: 26.2 s
Wall time: 27.6 s


#### The memory usage while loading csv and performing EDA(find the maximum rain drop)

| Approach        | memory usage(MB)           | execution time  |
| ------------- |:-------------:| -----:|
| Regular      | 3517.64 |   1min 10s    |
| Loading in chunks      | 2224.97      |  1min 1s  |
| Dask | 1019.96      |  1min 28s   |
| Load single column | 688.32      |  26.2 s   |

#### Observations
- To find the maximum rain drop, we only need one column in the table
- The approach of loading just this single column use the least memory and spend the shortest time
- Regular approach without any optimization use the most memory and spend the longest time

# 4. Save to various file formats

In [7]:
dataset = ds.dataset('figshare/combined_data.csv', format='csv')
arrow_table = dataset.to_table()

In [8]:
%%time
%%memit

feather.write_feather(arrow_table, 'figshare/combined_data.feather')

peak memory: 3242.37 MiB, increment: 2930.09 MiB
CPU times: user 5.07 s, sys: 8.96 s, total: 14 s
Wall time: 8.4 s


In [9]:
%%time
%%memit

pq.write_to_dataset(arrow_table, 'figshare/combined_data.parquet', partition_cols=['model'])

peak memory: 4656.05 MiB, increment: 1407.00 MiB
CPU times: user 23.5 s, sys: 20.9 s, total: 44.4 s
Wall time: 51.4 s


# 5. Perform a simple EDA in R

In [11]:
%%time
%%R
### her we are showing how much time it took to read a feather file what we wrote in python
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figshare/combined_data.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% select('rain (mm/day)') %>% max()
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "tbl_df"     "tbl"        "data.frame"
[1] 432.9395
Time difference of 26.65761 secs
CPU times: user 10 s, sys: 19.2 s, total: 29.3 s
Wall time: 26.8 s


#### Discussion
- Our team use `Feather file` approach to transfer the dataframe from python to R
- Our EDA(finding the maximum rain drop) only need a single column, so basically in Python we use Python Arrow to explore the data both for the space and time efficiency
- Feather is how we store the arrow table in memory to disk
- R has a good support on reading feather format data 